In [16]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
# This import is required to specify the local drawing method for the graph
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

In [17]:
class AgentState(TypedDict):

    number1: int
    number2: int
    number3: int
    number4: int
    operation1: str
    operation2: str
    finalnumber1: int
    finalnumber2: int

In [18]:
# --- Node Functions ---

def adder1(state: AgentState) -> AgentState:
    """This node adds the first two numbers."""
    print("---EXECUTING NODE: add1---")
    state['finalnumber1'] = state['number1'] + state['number2']
    return state

def adder2(state: AgentState) -> AgentState:
    """This node adds the second two numbers."""
    print("---EXECUTING NODE: add2---")
    state['finalnumber2'] = state['number3'] + state['number4']
    return state

def subtractor1(state: AgentState) -> AgentState:
    """This node subtracts the first two numbers."""
    print("---EXECUTING NODE: sub1---")
    state['finalnumber1'] = state['number1'] - state['number2']
    return state

def subtractor2(state: AgentState) -> AgentState:
    """This node subtracts the second two numbers."""
    print("---EXECUTING NODE: sub2---")
    # CORRECTED: Was using number1 and number2 by mistake.
    state['finalnumber2'] = state['number3'] - state['number4']
    return state

# --- Conditional Edge Functions ---

def decide_next_node1(state: AgentState) -> str:
    """Decides the first operation based on the 'operation1' key in the state."""
    print("---DECIDING: Operation 1---")
    if state["operation1"] == "+":
        # CORRECTED: Must return the actual node name.
        return "add1"
    elif state["operation1"] == "-":
        # CORRECTED: Must return the actual node name.
        return "sub1"

def decide_next_node2(state: AgentState) -> str:
    """Decides the second operation based on the 'operation2' key in the state."""
    print("---DECIDING: Operation 2---")
    if state["operation2"] == "+":
        # CORRECTED: Must return the actual node name.
        return "add2"
    elif state["operation2"] == "-":
        # CORRECTED: Must return the actual node name.
        return "sub2"

In [19]:
# --- Graph Definition ---

# Initialize the graph with the AgentState
graph = StateGraph(AgentState)

# Add all the nodes to the graph
graph.add_node("add1", adder1)
graph.add_node("sub1", subtractor1)
graph.add_node("add2", adder2)
graph.add_node("sub2", subtractor2)

# The "router" nodes are entry points for our conditional logic
graph.add_node("router1", lambda state: state)
graph.add_node("router2", lambda state: state)

# The graph starts and immediately goes to the first router
graph.add_edge(START, "router1")

# The first router decides which operation to perform
graph.add_conditional_edges(
    "router1",
    decide_next_node1,
    {
        # The keys here MUST match the return values of decide_next_node1
        "add1": "add1",
        "sub1": "sub1"
    }
)

# After the first operation, the flow goes to the second router
graph.add_edge("add1", "router2")
graph.add_edge("sub1", "router2")

# The second router decides which operation to perform
graph.add_conditional_edges(
    "router2",
    decide_next_node2,
    {
        # The keys here MUST match the return values of decide_next_node2
        "add2": "add2",
        "sub2": "sub2"
    }
)

# After the second operation, the graph ends
graph.add_edge("add2", END)
graph.add_edge("sub2", END)

# Compile the graph into a runnable application
app = graph.compile()

In [20]:
# --- Visualizing the Graph ---
# We'll try to render the graph locally using pyppeteer.
# You might need to install it first: pip install pyppeteer
try:
    print("Attempting to display graph visualization...")
    display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))
    print("Graph visualization displayed successfully.")
except Exception as e:
    print(f"\n---[GRAPH VISUALIZATION FAILED]---")
    print(f"Error displaying graph: {e}")
    print("\nCould not automatically render the graph. This might be because 'pyppeteer' is not installed or configured correctly.")
    print("As an alternative, you can copy the MermaidJS syntax below and paste it into a viewer like https://mermaid.live")
    print("\n--- MERMAID SYNTAX ---")
    # Print the Mermaid syntax as a fallback
    print(app.get_graph().draw_mermaid())
    print("----------------------\n")

Attempting to display graph visualization...

---[GRAPH VISUALIZATION FAILED]---
Error displaying graph: asyncio.run() cannot be called from a running event loop

Could not automatically render the graph. This might be because 'pyppeteer' is not installed or configured correctly.
As an alternative, you can copy the MermaidJS syntax below and paste it into a viewer like https://mermaid.live

--- MERMAID SYNTAX ---
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	add1(add1)
	sub1(sub1)
	add2(add2)
	sub2(sub2)
	router1(router1)
	router2(router2)
	__end__([<p>__end__</p>]):::last
	__start__ --> router1;
	add1 --> router2;
	router1 -.-> add1;
	router1 -.-> sub1;
	router2 -.-> add2;
	router2 -.-> sub2;
	sub1 --> router2;
	add2 --> __end__;
	sub2 --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

----------------------



C:\Users\Arjun Sarkar\AppData\Local\Temp\ipykernel_7184\3940246878.py:16: RuntimeWarning: coroutine '_render_mermaid_using_pyppeteer' was never awaited
  print("----------------------\n")


In [21]:
# --- Running the Graph ---

# Example 1: Add, then Subtract
print("---RUNNING EXAMPLE 1: Add -> Subtract---")
initial_state_1 = {
    "number1": 100, "number2": 50,
    "number3": 20, "number4": 8,
    "operation1": "+", "operation2": "-"
}
result_1 = app.invoke(initial_state_1)
print("\nFinal State 1:", result_1)
# Expected: finalnumber1 = 150, finalnumber2 = 12

print("\n" + "="*30 + "\n")

# Example 2: Subtract, then Add
print("---RUNNING EXAMPLE 2: Subtract -> Add---")
initial_state_2 = {
    "number1": 100, "number2": 50,
    "number3": 20, "number4": 8,
    "operation1": "-", "operation2": "+"
}
result_2 = app.invoke(initial_state_2)
print("\nFinal State 2:", result_2)
# Expected: finalnumber1 = 50, finalnumber2 = 28

---RUNNING EXAMPLE 1: Add -> Subtract---
---DECIDING: Operation 1---
---EXECUTING NODE: add1---
---DECIDING: Operation 2---
---EXECUTING NODE: sub2---

Final State 1: {'number1': 100, 'number2': 50, 'number3': 20, 'number4': 8, 'operation1': '+', 'operation2': '-', 'finalnumber1': 150, 'finalnumber2': 12}


---RUNNING EXAMPLE 2: Subtract -> Add---
---DECIDING: Operation 1---
---EXECUTING NODE: sub1---
---DECIDING: Operation 2---
---EXECUTING NODE: add2---

Final State 2: {'number1': 100, 'number2': 50, 'number3': 20, 'number4': 8, 'operation1': '-', 'operation2': '+', 'finalnumber1': 50, 'finalnumber2': 28}
